In [ ]:
# Libraries
import pandas as pd
import os
from ast import literal_eval

**Step 1:** db-creation  
- Firstly, we load two datasets (Python)

## Schema №1
We divide the automation in two domains:
1. DB-creation (reading & exporting to DB) using Python
2. DB-query (editing columns and joining tabels together) using SQL

In [ ]:
# Reading data

# XWines_Slim_1K_wines.csv
wines = pd.read_csv('XWines_Slim_1K_wines.csv',

                      dtype = {'WineID': int, 'WineName': str,
                               'Type': str, 'Elaborate':str,
                               'ABV': float, 'Body': str,
                               'Acidity':str, 'Code': str,
                               'Country': str, 'RegionID': int,
                               'RegionName': str, 'WineryID': int,
                               'Wineryname': str, 'Website': str},

                      converters = {'Grapes': literal_eval,
                                    'Harmonize':literal_eval,
                                    'Vintages': literal_eval})

# XWines_Slim_150K_ratings dataset
ratings = pd.read_csv('XWines_Slim_150K_ratings.csv',
                      sep = ',', parse_dates=['Date'])

<ipython-input-110-d3c73b8378af>:19: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings = pd.read_csv('XWines_Slim_150K_ratings.csv',


**Step 2:** db-creation
- Secondly, we edit datatype and delete redundant columns (Python)

In [ ]:
# Edit data type
ratings['WineID'] = ratings['WineID'].astype(int)

# Delete redundant columns
redund_cols = ['Code', 'Grapes', 'WineryID', 'Website']

# Create cleaned_wines frame
if any(col in wines.columns for col in redund_cols):
  cleaned_wines = wines.drop(redund_cols, axis=1)

**Step 3:** db-creation
- Thirdly, we create separate datasets & adding id for each (Python)
- Add joined_table

In [ ]:
# Creating frame for Regions
regions_db = cleaned_wines[['RegionID', 'RegionName']].drop_duplicates()

# Creating frame for Rating
rating_db = ratings[['RatingID', 'Rating']]

# Creating frame for Harmonize:
  # Unnest list-values
unnested_harmonize = cleaned_wines[['Harmonize', 'WineID']].explode('Harmonize')

  # Get unique harmonize-values
unique_harmonize = unnested_harmonize['Harmonize'].unique()

  # Create ID for unique harmonize
harmonize_db = pd.DataFrame()
harmonize_db['HarmonizeID'] = list(range(1000001, 1000001 + len(unique_harmonize)))
harmonize_db['Harmonize'] = unique_harmonize

  # Merge with WineID
harmonize_db = pd.merge(harmonize_db, unnested_harmonize, on = 'Harmonize')

# Creating frame for Wine:
wines_db = cleaned_wines

# Join: Region & Wine with Rating
joined_table_db = wines_db[['WineID', 'RegionID']] \
                .merge(ratings[['RatingID', 'WineID']], on='WineID')

# Join with Harmonize
joined_table_db = joined_table_db.merge(harmonize_db[['HarmonizeID', 'WineID']])

# Delete redundant column in harmonize_db & make uniq values
if 'WineID' in harmonize_db.columns:
  harmonize_db = harmonize_db.drop(columns = {'WineID'}).drop_duplicates()

---
**Step 1:** db-query (SQL)
- we calculate mean rating for each wine
- merge wine data & rating column

In [ ]:
# Join to get WineID
rating_db_wine = rating_db.merge(joined_table_db[['RatingID', 'WineID']], on = 'RatingID') \
                          .drop_duplicates()

# Calc mean rating
mean_rating = rating_db_wine.groupby(['WineID']).agg({'Rating': 'mean'}) \
                     .reset_index().round(2)

# Merge wine & ratings
wines_db = wines_db.merge(mean_rating, on = 'WineID')

**Step 2:** db-query (SQL)
- edit WineName values: adding Winery name to each wine

In [ ]:
# Combine wine name with its region acronym
wines_db['WineName'] = wines_db['WineName'] + ' (' + wines_db['WineryName'] + ')'

**Step 3:** db-query (SQL)
- We drop redundant columns

In [ ]:
# Delete redundant columns
redund_cols = ['RegionID', 'RegionName', 'WineryName']

if any(col in wines_db.columns for col in redund_cols):
  wines_db = wines_db.drop(redund_cols, axis=1)

**Step 4:** db-query (SQL)
- We delete brackets for Vintages columns

In [ ]:
wines_db['Vintages'] = wines_db['Vintages'].astype(str)
wines_db['Vintages'] = wines_db['Vintages'].str.replace(r"\[","")
wines_db['Vintages'] = wines_db['Vintages'].str.replace(r"\]","")

<ipython-input-97-af72363df3be>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  wines_db['Vintages'] = wines_db['Vintages'].str.replace(r"\[","")
<ipython-input-97-af72363df3be>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  wines_db['Vintages'] = wines_db['Vintages'].str.replace(r"\]","")


**Step 5:** db-query (SQL)
- We change dots with commas for Tableau

In [ ]:
# Additional for Tableau – change float types by comma
wines_db['ABV']= wines_db['ABV'].astype("string")
wines_db['Rating'] = wines_db['Rating'].astype('string')

wines_db['ABV'] = wines_db['ABV'].str.replace('.', ',', regex=True)
wines_db['Rating'] = wines_db['Rating'].str.replace('.', ',', regex=True)

---
## Schema №2
We divide the automation in two domains:
1. DB-creation (reading & exporting to DB) using Python
2. DB-query (editing columns and joining tabels together)

**Step 1:** db-creation  
- Firstly, we load two datasets (Python)

In [ ]:
# Reading data

# XWines_Slim_1K_wines.csv
wines = pd.read_csv('XWines_Slim_1K_wines.csv',

                      dtype = {'WineID': int, 'WineName': str,
                               'Type': str, 'Elaborate':str,
                               'ABV': float, 'Body': str,
                               'Acidity':str, 'Code': str,
                               'Country': str, 'RegionID': int,
                               'RegionName': str, 'WineryID': int,
                               'Wineryname': str, 'Website': str},

                      converters = {'Grapes': literal_eval,
                                    'Harmonize':literal_eval,
                                    'Vintages': literal_eval})

# XWines_Slim_150K_ratings dataset
ratings = pd.read_csv('XWines_Slim_150K_ratings.csv',
                      sep = ',', parse_dates=['Date'])

<ipython-input-99-d3c73b8378af>:19: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings = pd.read_csv('XWines_Slim_150K_ratings.csv',


**Step 2:** db-creation
- Secondly, we edit datatype and delete redundant columns (Python)

In [ ]:
# Edit data type
ratings['WineID'] = ratings['WineID'].astype(int)

# Delete redundant columns
redund_cols = ['Code', 'Grapes', 'WineryID', 'Website']

# Create cleaned_wines frame
if any(col in wines.columns for col in redund_cols):
  cleaned_wines = wines.drop(redund_cols, axis=1)

**Step 3:** db-creation
- Thirdly, we create separate datasets & adding id for each (Python)

In [ ]:
# Creating frame for Regions
regions_db = cleaned_wines[['RegionID', 'RegionName', 'WineID']]

# Creating frame for Rating
rating_db = ratings[['RatingID', 'WineID', 'Rating']]


# Creating frame for Harmonize:
  # Unnest list-values
unnested_harmonize = cleaned_wines[['Harmonize', 'WineID']].explode('Harmonize')

  # Get unique harmonize-values
unique_harmonize = unnested_harmonize['Harmonize'].unique()

  # Create ID for unique harmonize
harmonize_db = pd.DataFrame()
harmonize_db['HarmonizeID'] = list(range(1000001, 1000001 + len(unique_harmonize)))
harmonize_db['Harmonize'] = unique_harmonize

  # Merge with WineID
harmonize_db = pd.merge(harmonize_db, unnested_harmonize, on = 'Harmonize')

# Creating frame for Wine:
wines_db = cleaned_wines

---
**Step 1:** db-query (SQL)
- we calculate mean rating for each wine
- merge wine data & rating column

In [ ]:
# Calc mean rating
mean_rating = rating_db.groupby(['WineID']).agg({'Rating': 'mean'}) \
                     .reset_index().round(2)

# Merge wine & ratings
wines_db = wines_db.merge(mean_rating, on = 'WineID')

**Step 2:** db-query (SQL)
- edit WineName values: adding Winery name to each wine

In [ ]:
# Combine wine name with its region acronym
wines_db['WineName'] = wines_db['WineName'] + ' (' + wines_db['WineryName'] + ')'

**Step 3:** db-query (SQL)
- We drop redundant columns

In [ ]:
# Delete redundant columns
redund_cols = ['RegionID', 'RegionName', 'WineryName']

if any(col in wines_db.columns for col in redund_cols):
  wines_db = wines_db.drop(redund_cols, axis=1)

**Step 4:** db-query (SQL)
- We delete brackets for Vintages columns

In [ ]:
wines_db['Vintages'] = wines_db['Vintages'].astype(str)
wines_db['Vintages'] = wines_db['Vintages'].str.replace(r"\[","")
wines_db['Vintages'] = wines_db['Vintages'].str.replace(r"\]","")

<ipython-input-105-af72363df3be>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  wines_db['Vintages'] = wines_db['Vintages'].str.replace(r"\[","")
<ipython-input-105-af72363df3be>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  wines_db['Vintages'] = wines_db['Vintages'].str.replace(r"\]","")


**Step 5:** db-query (SQL)
- We change dots with commas for Tableau

In [ ]:
# Additional for Tableau – change float types by comma
wines_db['ABV']= wines_db['ABV'].astype("string")
wines_db['Rating'] = wines_db['Rating'].astype('string')

wines_db['ABV'] = wines_db['ABV'].str.replace('.', ',', regex=True)
wines_db['Rating'] = wines_db['Rating'].str.replace('.', ',', regex=True)

In [ ]:
# Фото добавить и все готово